In [ ]:
# import os

# # CUDA 디바이스 0, 2만 사용하도록 설정
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from dataclasses import dataclass

import torch
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
@dataclass
class Config:
    output_dir: str = 'kanana21'
    checkpoint: str = "kakaocorp/kanana-1.5-2.1b-instruct-2505"   # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 20
    fold_idx: int = 0
    optim_type: str = "adamw_torch"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 4  # global batch size is 16
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 0  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 1e-4
    warmup_steps: int = 20
    lora_r: int = 64
    lora_alpha: float = 16
    lora_dropout: float = 0.
    lora_bias: str = "none"

config = Config()

In [ ]:
training_args = TrainingArguments(
    output_dir=config.output_dir,
    do_eval=True,
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="steps",
    save_total_limit=3,
    save_steps=200,
    optim=config.optim_type,
    weight_decay=1e-2,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    dataloader_num_workers=64
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
tokenizer.padding_side = 'right'
tokenizer.truncation_side='right'
tokenizer.add_eos_token = True

In [ ]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,                    # ✅ 8bit quantization 사용
    llm_int8_skip_modules=["score"]       # quantization 제외할 모듈
)

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
    modules_to_save=["score"]
)

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=2,
    #quantization_config=bnb_config,
    device_map="auto",
    ignore_mismatched_sizes=True,
)

model.config.use_cache = False
#model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("./stage1_kanana")

In [ ]:
ds = ds.map(lambda x: {'labels': [x['labels'], x['generated']]})

In [ ]:
# Step 1: 'generated' 컬럼 제거
ds = ds.remove_columns(['generated'])

# Step 2: 컬럼 이름 변경
ds = ds.rename_columns({
    "original_input_ids": "input_ids",
    "original_attention_mask": "attention_mask",
})

In [ ]:
ds

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import torch
from transformers import EvalPrediction
import numpy as np

def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # shape: (batch_size, 2)
    labels = eval_preds.label_ids   # shape: (batch_size, 2)

    probs = torch.from_numpy(preds).float().sigmoid().numpy()  # shape: (batch_size, 2)
    binary_preds = (probs >= 0.5).astype(int)

    results = {}
    task_names = ["labels", "generated"]

    for i, task in enumerate(task_names):
        y_true = labels[:, i]
        y_prob = probs[:, i]
        y_pred = binary_preds[:, i]

        results[f"{task}_log_loss"] = log_loss(y_true, y_prob)
        results[f"{task}_auc"] = roc_auc_score(y_true, y_prob)
        results[f"{task}_accuracy"] = accuracy_score(y_true, y_pred)
        break
    return results

In [ ]:
from math import ceil

def get_train_val_split_indices(dataset, n_splits):
    total = len(dataset)
    split_size = ceil(total / n_splits)
    indices = list(range(total))

    # Split into n_splits parts
    splits = [indices[i*split_size:(i+1)*split_size] for i in range(n_splits)]

    # Train: all but last split
    train_indices = [i for split in splits[:-1] for i in split]
    val_indices = splits[-1]

    return train_indices, val_indices

# 예시 사용:
train_idx, eval_idx = get_train_val_split_indices(ds, config.n_splits)

In [ ]:
from transformers import Trainer
import torch.nn as nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")  # shape: (batch_size, 2)
        outputs = model(**inputs)
        logits = outputs.get("logits")  # shape: (batch_size, 2)

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id


In [ ]:

trainer = CustomTrainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

In [ ]:
ds = Dataset.from_csv("test.csv")

In [ ]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        title = ["<Title>: " + t for t in batch["title"]]
        para = ["\n\n<Full text>: " + t for t in batch["paragraph_text"]]
        texts = [t + p for t, p in zip(title, para)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)

        return {**tokenized}

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=1024)
ds = ds.map(encode, batched=True)

In [ ]:
from tqdm.notebook import tqdm
from transformers.data.data_collator import pad_without_fast_tokenizer_warning


@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(ds, model, batch_size=1):
    preds = []
    pseudo = []
    model.eval()
    
    for start_idx in tqdm(range(0, len(ds), batch_size)):
        end_idx = min(start_idx + batch_size, len(ds))
        tmp = ds[start_idx:end_idx]
        input_ids = tmp["input_ids"]
        attention_mask = tmp["attention_mask"]
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to("cuda:0"))
        proba = outputs.logits.cpu()
        
        preds.extend(proba[:, 0].tolist())
        pseudo.extend(proba[:,1].tolist())
    
    return preds, pseudo

In [ ]:
a, b = inference(ds, model)

In [ ]:
import pandas as pd


sub = pd.read_csv('sample_submission.csv')
sub.head()

sub['generated'] = a

sub.to_csv('stage2_a_kanana21b.csv', index=False)